In [17]:
from scipy.io import arff
import pandas as pd
import numpy as np
import os
from som_toolbox import somtoolbox
from som_toolbox.SOMToolBox_Parse import SOMToolBox_Parse
from som_toolbox.somtoolbox import SOMToolbox
from minisom import MiniSom
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

In [18]:
# Load the dataset
data_path = os.path.join("data")
data = arff.loadarff(os.path.join(data_path, "credit-g.arff"))

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(data[0])

# Decode byte strings in object columns
for col in df.select_dtypes([object]).columns:
    df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

In [3]:
df

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,female div/dep/mar,none,...,real estate,31.0,none,own,1.0,unskilled resident,1.0,none,yes,good
996,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,male div/sep,none,...,life insurance,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,good
997,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,male single,none,...,car,38.0,none,own,1.0,skilled,1.0,none,yes,good
998,<0,45.0,existing paid,radio/tv,1845.0,<100,1<=X<4,4.0,male single,none,...,no known property,23.0,none,for free,1.0,skilled,1.0,yes,yes,bad


In [4]:
df.dtypes

checking_status            object
duration                  float64
credit_history             object
purpose                    object
credit_amount             float64
savings_status             object
employment                 object
installment_commitment    float64
personal_status            object
other_parties              object
residence_since           float64
property_magnitude         object
age                       float64
other_payment_plans        object
housing                    object
existing_credits          float64
job                        object
num_dependents            float64
own_telephone              object
foreign_worker             object
class                      object
dtype: object

transform categorical columns to numeric using manual mapping

In [19]:
# First, examine all categorical columns and their unique values
print("=== Categorical Columns and Unique Values ===\n")
categorical_cols = df.select_dtypes([object]).columns.tolist()
for col in categorical_cols:
    unique_vals = sorted(df[col].unique())
    print(f"{col}: {unique_vals}\n")

# Encode categorical variables to numeric with ordinal mappings where applicable
df_encoded = df.copy()

# Define ordinal mappings (preserving meaningful order)
ordinal_mappings = {
    # Checking status: worst (negative) to best
    'checking_status': {
        '<0': 0,              # Negative balance - worst
        '0<=X<200': 1,        # Low positive balance
        '>=200': 2,           # Good balance
        'no checking': 3      # No checking account - can be best or neutral
    },
    
    # Savings status: lowest to highest savings
    'savings_status': {
        'no known savings': 0,    # No savings - worst
        '<100': 1,                # Very low savings
        '100<=X<500': 2,          # Low savings
        '500<=X<1000': 3,         # Moderate savings
        '>=1000': 4               # High savings - best
    },
    
    # Employment: worst (unemployed) to best (long-term employed)
    'employment': {
        'unemployed': 0,          # Unemployed - worst
        '<1': 1,                  # Less than 1 year
        '1<=X<4': 2,              # 1 to 4 years
        '4<=X<7': 3,              # 4 to 7 years
        '>=7': 4                  # 7+ years - best (most stable)
    },
    
    # Property magnitude: value from lowest to highest (note: column name is 'property_magnitude')
    'property_magnitude': {
        'no known property': 0,   # No property - worst
        'car': 1,                 # Car (moderate value)
        'life insurance': 2,      # Life insurance (better)
        'real estate': 3          # Real estate - best (highest value)
    },
    
    # Job quality: worst to best (FIXED: actual value is 'unemp/unskilled non res')
    'job': {
        'unemp/unskilled non res': 0,   # Worst - FIXED
        'unskilled resident': 1,                   # Low skill
        'skilled': 2,                              # Skilled worker
        'high qualif/self emp/mgmt': 3             # Best (management/high qualification)
    },
    
    # Credit history: worst (critical) to best (all paid, no credits) (FIXED: actual value is 'no credits/all paid')
    'credit_history': {
        'critical/other existing credit': 0,       # Critical - worst
        'existing paid': 1,                        # Paid but existing
        'delayed previously': 2,                   # Had delays
        'all paid': 3,                             # All paid
        'no credits/all paid': 4                   # Best - no previous issues (FIXED)
    },
    
    # Personal status: ordered by typical financial stability
    'personal_status': {
        'male single': 0,
        'female div/dep/mar': 1,
        'male div/sep': 2,
        'male mar/wid': 3         # Married/divorced/widowed - might be more stable
    },
    
    # Purpose: ordered by typical credit risk (FIXED: 'domestic appliance' singular, added 'retraining')
    'purpose': {
        'education': 0,
        'furniture/equipment': 1,
        'radio/tv': 2,
        'domestic appliance': 3,      # FIXED: singular form
        'retraining': 4,              # ADDED: retraining
        'repairs': 5,
        'used car': 6,
        'new car': 7,
        'business': 8,
        'other': 9
    },
    
    # Other debtors/guarantors: ordered by level of support (FIXED: 'co applicant' without hyphen)
    'other_parties': {
        'none': 0,                 # No support
        'guarantor': 1,            # Has guarantor (better)
        'co applicant': 2          # Co-applicant (best support) - FIXED: no hyphen
    },
    
    # Housing: ordered by stability/ownership
    'housing': {
        'for free': 0,             # Free housing (unclear stability)
        'rent': 1,                 # Renting
        'own': 2                   # Owning - best (most stable)
    },
    
    # Other payment plans: ordered by risk level
    'other_payment_plans': {
        'none': 0,                 # No other plans
        'stores': 1,               # Store credit
        'bank': 2                  # Bank credit (might be higher commitment)
    },
    
    # Binary variables (yes/no) - FIXED: column name is 'own_telephone', not 'telephone'
    'own_telephone': {'none': 0, 'yes': 1},  # FIXED: column name and values
    'foreign_worker': {'no': 0, 'yes': 1},
    
    # Target variable
    'class': {'bad': 0, 'good': 1}
}

# Apply ordinal mappings to columns that have them defined
print("\n=== Applying Ordinal Mappings ===\n")
for col, mapping in ordinal_mappings.items():
    if col in df_encoded.columns:
        print(f"Mapping {col}: {sorted(mapping.keys())}")
        df_encoded[col] = df_encoded[col].map(mapping)
        # Check for any unmapped values (useful for debugging)
        if df_encoded[col].isna().any():
            unmapped_vals = df[df_encoded[col].isna()][col].unique()
            print(f"  WARNING: Unmapped values in {col}: {unmapped_vals}")

# For any remaining categorical columns (shouldn't be any if mappings are complete)
# Use label encoding as fallback
remaining_categorical = df_encoded.select_dtypes([object]).columns.tolist()
if remaining_categorical:
    print(f"\n=== Using Label Encoding for remaining columns: {remaining_categorical} ===\n")
    label_encoders = {}
    for col in remaining_categorical:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le

print("\n=== Encoding Complete ===\n")
print(f"Encoded dataframe shape: {df_encoded.shape}")
print(f"Encoded dataframe dtypes:\n{df_encoded.dtypes.value_counts()}")

=== Categorical Columns and Unique Values ===

checking_status: ['0<=X<200', '<0', '>=200', 'no checking']

credit_history: ['all paid', 'critical/other existing credit', 'delayed previously', 'existing paid', 'no credits/all paid']

purpose: ['business', 'domestic appliance', 'education', 'furniture/equipment', 'new car', 'other', 'radio/tv', 'repairs', 'retraining', 'used car']

savings_status: ['100<=X<500', '500<=X<1000', '<100', '>=1000', 'no known savings']

employment: ['1<=X<4', '4<=X<7', '<1', '>=7', 'unemployed']

personal_status: ['female div/dep/mar', 'male div/sep', 'male mar/wid', 'male single']

other_parties: ['co applicant', 'guarantor', 'none']

property_magnitude: ['car', 'life insurance', 'no known property', 'real estate']

other_payment_plans: ['bank', 'none', 'stores']

housing: ['for free', 'own', 'rent']

job: ['high qualif/self emp/mgmt', 'skilled', 'unemp/unskilled non res', 'unskilled resident']

own_telephone: ['none', 'yes']

foreign_worker: ['no', 'yes']


In [6]:
df_encoded

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,0,6.0,0,2,1169.0,0,4,4.0,0,0,...,3,67.0,0,2,2.0,2,1.0,1,1,1
1,1,48.0,1,2,5951.0,1,2,2.0,1,0,...,3,22.0,0,2,1.0,2,1.0,0,1,0
2,3,12.0,0,0,2096.0,1,3,2.0,0,0,...,3,49.0,0,2,1.0,1,2.0,0,1,1
3,0,42.0,1,1,7882.0,1,3,2.0,0,1,...,2,45.0,0,0,1.0,2,2.0,0,1,1
4,0,24.0,2,7,4870.0,1,2,3.0,0,0,...,0,53.0,0,0,2.0,2,2.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,12.0,1,1,1736.0,1,3,3.0,1,0,...,3,31.0,0,2,1.0,1,1.0,0,1,1
996,0,30.0,1,6,3857.0,1,2,4.0,2,0,...,2,40.0,0,2,1.0,3,1.0,1,1,1
997,3,12.0,1,2,804.0,1,4,4.0,0,0,...,1,38.0,0,2,1.0,2,1.0,0,1,1
998,0,45.0,1,2,1845.0,1,2,4.0,0,0,...,0,23.0,0,0,1.0,2,1.0,1,1,0


In [7]:
# check nas
df_encoded.isna().sum()

checking_status           0
duration                  0
credit_history            0
purpose                   0
credit_amount             0
savings_status            0
employment                0
installment_commitment    0
personal_status           0
other_parties             0
residence_since           0
property_magnitude        0
age                       0
other_payment_plans       0
housing                   0
existing_credits          0
job                       0
num_dependents            0
own_telephone             0
foreign_worker            0
class                     0
dtype: int64

In [20]:
# Convert to numpy array for minisom
data_array = df_encoded.values.astype(float)

# Scale the data to [0, 1] range
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_array)

# Calculate dataset size for SOM dimension recommendation
n_samples = data_scaled.shape[0]
recommended_neurons = int(5 * np.sqrt(n_samples))
print(f"Dataset size: {n_samples} samples")
print(f"Recommended SOM size (5*sqrt(N)): ~{recommended_neurons} neurons")

# SOM Parameters
# Rule of thumb: 5*sqrt(N) neurons, so for ~1000 samples: ~158 neurons
# Options: 10x10=100 (conservative), 12x12=144 (balanced), 15x15=225 (detailed)
SOM_X_AXIS_NODES = 12  # Increased from 8 for better representation
SOM_Y_AXIS_NODES = 12  # Increased from 8 for better representation
SOM_N_VARIABLES = data_scaled.shape[1]

# Learning parameters
ALPHA = 0.7  # Initial learning rate
SIGMA0 = 3.0  # Initial neighborhood radius (30-50% of map size: 12*0.25-0.5 = 3-6)
NEIGHBORHOOD_FUNC = 'gaussian' 
N_ITERATIONS = 10000 

# Create SOM with parameters
som = MiniSom(SOM_X_AXIS_NODES, SOM_Y_AXIS_NODES, SOM_N_VARIABLES,
              sigma=SIGMA0,
              learning_rate=ALPHA,
              neighborhood_function=NEIGHBORHOOD_FUNC,
              random_seed=42)

# Initialize weights using PCA
som.pca_weights_init(data_scaled)

# Train the SOM
som.train_random(data_scaled, N_ITERATIONS, verbose=True)

print(f"\nSOM Training Complete!")
print(f"Quantization error: {som.quantization_error(data_scaled):.4f}")
print(f"Topographic error: {som.topographic_error(data_scaled):.4f}")  

# Reshape weights from (m, n, dimension) to (m*n, dimension) for SOMToolbox
weights_3d = som.get_weights()  # Shape: (m, n, dimension)
weights_2d = weights_3d.reshape(SOM_X_AXIS_NODES * SOM_Y_AXIS_NODES, SOM_N_VARIABLES)  # Shape: (m*n, dimension)

# Convert classes to numpy array
classes_array = df_encoded['class'].values if 'class' in df_encoded.columns else None

# Convert component_names to list (not numpy array) to avoid boolean evaluation issues
component_names_list = df_encoded.columns.tolist()

sm = SOMToolbox(weights=weights_2d, m=SOM_X_AXIS_NODES, n=SOM_Y_AXIS_NODES,
            dimension=SOM_N_VARIABLES, input_data=data_scaled,
            classes=classes_array, component_names=component_names_list)
sm._mainview

Dataset size: 1000 samples
Recommended SOM size (5*sqrt(N)): ~158 neurons


/Users/janwika/miniconda3/envs/SOS2025/lib/python3.11/site-packages/minisom.py:447: ComplexWarning: Casting complex values to real discards the imaginary part
  self._weights[i, j] = c1*pc[pc_order[0]] + \


 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 0.9723502067101895

SOM Training Complete!
Quantization error: 0.9724
Topographic error: 0.0630


BokehModel(combine_events=True, render_bundle={'docs_json': {'df3a3d9f-6f76-4fa6-843c-3da149c15c94': {'version…

## 1 Identify border effects and magnification factors

Visualization to use: Compare the U-Matrix with the Frequency (Hit) Map.
U-Matrix:
Looking at the U-Matrix, there is evidence of border effects, particularly on the left edge (around y=0).
Observation: There is a distinct high-distance cluster (green/red spikes) right at (-0.5, 0).
This suggests the map is stretching significantly at the boundary to accommodate outlier data points that lie far outside the main distribution. The "walls" (high U-matrix values) seem to press against the edges of the map rather than being fully contained within it.

Magnification Factors:

Comparing the U-Matrix and the Hit Histogram (Top Right):
Spots at (-0.5, -0.5) and (0.5, -0.5) are very bright, indicating high hit counts -> high magnification in those areas
U matrix shows low values there -> 


U-Matrix with pie overlay-> 
not that well separated, but we see most of the positive class datapoints in bottom left with high distances
and some of them on middle left and some on top -> ie not really together
only bottom left majority of positive class, other areas quite mixed

QE not evenly distributed, concentrated at bottom left and middle left
-> overlaps with hit histogram -> potential sign of underfitting -> potentially more neurons needed or learning rate decay too fast


# 2 Different random seed

In [21]:
# Create SOM with parameters
som2 = MiniSom(SOM_X_AXIS_NODES, SOM_Y_AXIS_NODES, SOM_N_VARIABLES,
              sigma=SIGMA0,
              learning_rate=ALPHA,
              neighborhood_function=NEIGHBORHOOD_FUNC,
              random_seed=123)

# Initialize weights using PCA
som2.pca_weights_init(data_scaled)

# Train the SOM
som2.train_random(data_scaled, N_ITERATIONS, verbose=True)

print(f"\nSOM Training Complete!")
print(f"Quantization error: {som2.quantization_error(data_scaled):.4f}")
print(f"Topographic error: {som2.topographic_error(data_scaled):.4f}")  

# Reshape weights from (m, n, dimension) to (m*n, dimension) for SOMToolbox
weights_3d = som2.get_weights()  # Shape: (m, n, dimension)
weights_2d = weights_3d.reshape(SOM_X_AXIS_NODES * SOM_Y_AXIS_NODES, SOM_N_VARIABLES)  # Shape: (m*n, dimension)

# Convert classes to numpy array
classes_array = df_encoded['class'].values if 'class' in df_encoded.columns else None

# Convert component_names to list (not numpy array) to avoid boolean evaluation issues
component_names_list = df_encoded.columns.tolist()

sm = SOMToolbox(weights=weights_2d, m=SOM_X_AXIS_NODES, n=SOM_Y_AXIS_NODES,
            dimension=SOM_N_VARIABLES, input_data=data_scaled,
            classes=classes_array, component_names=component_names_list)
sm._mainview

/Users/janwika/miniconda3/envs/SOS2025/lib/python3.11/site-packages/minisom.py:447: ComplexWarning: Casting complex values to real discards the imaginary part
  self._weights[i, j] = c1*pc[pc_order[0]] + \


 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 0.9746838270929756

SOM Training Complete!
Quantization error: 0.9747
Topographic error: 0.0720


BokehModel(combine_events=True, render_bundle={'docs_json': {'b27af00e-49bf-461b-a567-03f3c636abdb': {'version…

# 2 Jan

In [22]:
# --- 1. Train SOM 2 (Random Initialization) ---
print("Training SOM 2 with Random Seed...")

# Same parameters as your first map
som2 = MiniSom(SOM_X_AXIS_NODES, SOM_Y_AXIS_NODES, SOM_N_VARIABLES,
              sigma=SIGMA0,
              learning_rate=ALPHA,
              neighborhood_function=NEIGHBORHOOD_FUNC,
              random_seed=123) # <--- Different seed

# CRITICAL: Random Init (instead of PCA) to force differences
som2.random_weights_init(data_scaled)
som2.train_random(data_scaled, N_ITERATIONS, verbose=True)

# --- 2. Prepare for SOMToolbox ---
# Prepare Map 1 (The PCA one you already have)
weights_1 = som.get_weights().reshape(SOM_X_AXIS_NODES * SOM_Y_AXIS_NODES, SOM_N_VARIABLES)
sm1 = SOMToolbox(weights=weights_1, m=SOM_X_AXIS_NODES, n=SOM_Y_AXIS_NODES,
                 dimension=SOM_N_VARIABLES, input_data=data_scaled,
                 classes=classes_array, component_names=component_names_list)

# Prepare Map 2 (The new Random one)
weights_2 = som2.get_weights().reshape(SOM_X_AXIS_NODES * SOM_Y_AXIS_NODES, SOM_N_VARIABLES)
sm2 = SOMToolbox(weights=weights_2, m=SOM_X_AXIS_NODES, n=SOM_Y_AXIS_NODES,
                 dimension=SOM_N_VARIABLES, input_data=data_scaled,
                 classes=classes_array, component_names=component_names_list)

sm2._mainview

Training SOM 2 with Random Seed...
 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 0.9759215851192371


BokehModel(combine_events=True, render_bundle={'docs_json': {'bab4df2e-9053-4ae7-be5f-75fc0863c76a': {'version…

- Train one further „regular-sized“ SOM using the same training parameters as above, but using
a different random seed for initializing the SOM.
- Show and describe a) how the cluster structures and class distributions shift on the two
SOMs, b) the effect on topology violations, cluster relationships, etc. c) Which clusters show
a stable relationship, which ones change their relative position? d) Which data instances are
stably mapped with similar data instances, which change a lot? Are they part of the same
clusters?
- Describe and compare the structures found (providing detailed info on visualizations and
parameters)

a.) shift of cluster structures / distributions
Visualizations Used: U-matrix with pie charts
The clusters in the randomly initialized SOM form quite differently than the clusters in the PCA initialized SOM. While there is still a horizontal border at y=0 there are more and smaller clusters. The biggest cluster is situated at the bottom left as opposed to bottom right for the PCA initialized SOM. Above this cluster is the only cluster that has not situated at the edges of the graph but rather surrounded by borders. There is no obvious transformation like for example rotation that could be applied to the U-matrix of the PCA initialized SOM to get a similar looking U_matrix to this one. These observations suggest that the random initialization has a significant impact on the SOM. Looking at the pie chart overlayed U-matrix reveals that the 0 labeled (bad) data points still largely don't fall into clusters but on borders, however they center around two points in the middle-right and top-right rather than one point  bottom-left for the PCA initialized SOM.

b.) topology violations
Visualizations Used: Neighbourhood Map (KNN, k = 1), Quantization Error
The randomly initialized map shows more long, crossing lines in the Neighborhood Graph compared to the other map (PCA), which suggests that the random initialization led to a 'twisted' map with higher topology violations. Although the PCA map exhibits quite chaotic neoghbourhood lines it is even more pronounced in the randomly intialized one. The quantization error matrices don't share similiarities.

c.) stable relationships
Visualizations Used: Component Planes
The component planes of most features in the randomly intialized SOM bared little resemblence to their corresponding visualizations in the PCA initialized SOM. However the features 8 (personal_status), 9 (other_parties), 17 (num-dependents) and 18 (own_telephone) looked like linear transformations of one another. This is most pronounced in feature 18. In the PCA initialized SOM there is a triangle of red values taking up roughly a third of the matrix in the top-right and a red dot in the bottom-left, separated by a blue center. For the randomly initialized SOM the size proportions of the two shapes are more equal but are still resembling their counterpart when rotated 90° to the right. This suggests that while in absolute coordinates the clusters are arbitrary, the relative topology of the mentioned features remains somewhat stable indicating that both SOMs captured some intrinsic correlations.

d.) stable instances
Visualizations Used: Hit Histogram with pie-charts
In both the randomly initialized and PCA initialized SOMs there are units with high hit numbers and a large portion of the minority class being mapped there. However they are situated at entirely different coordinates bottom-left for PCA and center-right / top-right for random. Situated along the borders (from U-matrix) are units with mixed pie charts for both SOMs although the borders themselves are at different coordinates.

## 3) Analyze different map sizes:
- Train 2 additional SOMs varying the size (very small / very large) (provide reasons for choice
of sizes)
- Train each map with rather large neighborhood radius and high learning rate (provide reasons
for the definition of „high“!)
- Analyse in detail the a) class distribution, b) cluster structure, c) quantization errors, d)
topology violations. Also, e) analyze how clusters shift, change in relative size, and how their
relative position to each other changes or remains the same. f) Check for aspects such as
magnification factors. What is the resulting granularity of clusters visible on the small and large
maps? Are the same clusters visible in the very large map as in the regular map?
- Describe and compare the structures found (providing detailed info on visualizations and
parameters)

In [58]:
# --- Parameters ---
# 1. Sizes
SMALL_DIM = 5    # 5x5 = 25 units
LARGE_DIM = 25   # 25x25 = 625 units

# 2. Training Hyperparameters (High LR, Large Sigma)
LR_HIGH = 0.8           # Aggressive starting rate for ordering
ITERATIONS = 10000

# --- Train SMALL Map ---
print(f"Training SMALL Map ({SMALL_DIM}x{SMALL_DIM})...")
# Sigma is large relative to map size (covering ~50% of map)
sigma_small = SMALL_DIM / 2.0 

som_small = MiniSom(SMALL_DIM, SMALL_DIM, data_scaled.shape[1],
                    sigma=sigma_small, learning_rate=LR_HIGH,
                    neighborhood_function='gaussian', random_seed=42)
som_small.pca_weights_init(data_scaled)
som_small.train_random(data_scaled, ITERATIONS, verbose=True)

# --- Train LARGE Map ---
print(f"\nTraining LARGE Map ({LARGE_DIM}x{LARGE_DIM})...")
# Sigma must be scaled up for the large map to be proportionally "large"
sigma_large = LARGE_DIM / 2.0 

som_large = MiniSom(LARGE_DIM, LARGE_DIM, data_scaled.shape[1],
                    sigma=sigma_large, learning_rate=LR_HIGH,
                    neighborhood_function='gaussian', random_seed=42)
som_large.pca_weights_init(data_scaled)
som_large.train_random(data_scaled, ITERATIONS, verbose=True)

# --- Metrics & Export ---
print(f"\nSmall Map QE: {som_small.quantization_error(data_scaled):.4f} | TE: {som_small.topographic_error(data_scaled):.4f}")
print(f"Large Map QE: {som_large.quantization_error(data_scaled):.4f} | TE: {som_large.topographic_error(data_scaled):.4f}")

# Prepare Small Map for Toolbox
w_small = som_small.get_weights().reshape(SMALL_DIM * SMALL_DIM, data_scaled.shape[1])
sm_small = SOMToolbox(weights=w_small, m=SMALL_DIM, n=SMALL_DIM,
                      dimension=data_scaled.shape[1], input_data=data_scaled,
                      classes=classes_array, component_names=component_names_list)

# Prepare Large Map for Toolbox
w_large = som_large.get_weights().reshape(LARGE_DIM * LARGE_DIM, data_scaled.shape[1])
sm_large = SOMToolbox(weights=w_large, m=LARGE_DIM, n=LARGE_DIM,
                      dimension=data_scaled.shape[1], input_data=data_scaled,
                      classes=classes_array, component_names=component_names_list)

Training SMALL Map (5x5)...


/Users/janwika/miniconda3/envs/SOS2025/lib/python3.11/site-packages/minisom.py:447: ComplexWarning: Casting complex values to real discards the imaginary part
  self._weights[i, j] = c1*pc[pc_order[0]] + \


 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 1.181561370644542

Training LARGE Map (25x25)...
 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 1.102536572234628

Small Map QE: 1.1816 | TE: 0.1530
Large Map QE: 1.1025 | TE: 0.0140


In [60]:
sm_small._mainview

BokehModel(combine_events=True, render_bundle={'docs_json': {'f3951857-ca2c-4981-8897-4cf4fd40d86e': {'version…

In [61]:
sm_large._mainview

BokehModel(combine_events=True, render_bundle={'docs_json': {'1d695623-3167-4b80-9a45-14dbfc0da53e': {'version…

3) Analyze different map sizes:
- Train 2 additional SOMs varying the size (very small / very large) (provide reasons for choice
of sizes)
- Train each map with rather large neighborhood radius and high learning rate (provide reasons
for the definition of „high“!)
- Analyse in detail the a) class distribution, b) cluster structure, c) quantization errors, d)
topology violations. Also, e) analyze how clusters shift, change in relative size, and how their
relative position to each other changes or remains the same. f) Check for aspects such as
magnification factors. What is the resulting granularity of clusters visible on the small and large
maps? Are the same clusters visible in the very large map as in the regular map?
- Describe and compare the structures found (providing detailed info on visualizations and
parameters)

A high initial learning rate (0.8, lecture slides classify 0.7 - 0.9 as high)is selected to maximize plasticity during the ordering phase, ensuring the map quckly expands to cover the data distribution rather than getting stuck in local minima. A large neighborhood radius is used to force the grd to move as a coherent unit, thereby establishing global topological order and preventing the formation of twists or folds. The 

a.) class distribution
Visualizations Used: Pie Charts
For both SOMs it can be observed that most data points are mapped to the edges of the matrix. For the small SOM there is one pie charts that indicates mostly samples of the minority class are mapped to the corresponding unit (center-bottom). Otherwise the minority class is spread to other units where less than a fourth of the mapped data points are of this class apart from the unit in the top-left wher 3/4th are of the minority class. This indicates that while some units (2) capture the correllations well, in many other cases the SOM is "underfitting" meaning due to a lack of "resolution" datapoints are bucketed together although they shouldn't be. It is the opposite case for the large SOM. Large parts of the map don't have any or very small overlayed pie charts indicating that vast stretches of the SOM are not or under- untilized.

b.) cluster structure
Visualizations Used: U-Matrix
Clusters are hard to make out in the 5x5 SOM since the reslution is so low. However there appeear to be clusters in the bottom-left/center and top-left/center which are also visible in the large SOM. There is a large border in the center of the large SOM which is not visible in the small SOM, which appears to have a border in the bottom-right. When looking at the U-matrix of the large SOM there is a quite obvious checkerboard pattern visible which is most prevalent at the borders of clusters. This is a sign of sparsity indicating the SOM is too large for the dataset. Especially due to the large learning rate and neighbourhood the map is too stretched out. 

c.) quantization errors
Visualizations Used: Quantization Error
At first glance the quantization error seems to be very low for the large SOM. However, since most datapoints are mapped to a small portion of the map this conclusion cannot be drawn. In all corners, especially bottom-left QE is high and this is where most datapoints are mapped to. Also the class imbalance has to be considered. The even higher QE in the small SOM can be seen more easily in the map. There are clear red areas in the bottom-left and bottom-right. Total QE for the small SOM is high at 1.1816 but the large SOM doesn't perform much better with 1.1025. It is important to note here that the range of quantization error is also dependent on the number of input features which is quite high (19).

d.) topology violations
Visualizations Used: Neighbourhood Graph
For the small SOM the neighbourhood graph almost resembles a fully connected graph even for KNN with k=1 with more sparsity in the lower-right graph. The larger graph on the other hand has more densly connected areas and others that are more sparse. However, even the large SOM's neighbourhood graph reveals quite a few "knots" and long edges indicating topology violations to a certain degree.

e.) cluster shifts
explained in point b

f.) magnification factor
Visualizations Used: Component Plane
A magnification effect can be seen quite clearly when observing the component plane for feature 18 (own_telephone). While the large SOMs compoennt plane is shifted 90° degrees to the left compared to the component plane of the small SOM there is a clear gradient visible from bottom/left (high) to top/right (low). The difference between the large and the small SOM lies mostly in reolution. While the small SOM's component map provides a step wise gradient, the one of the large SOM is more smooth.

## 4) Analyze different initial neighborhood radius settings:
- Train the very large SOM as specified above, but with a much too small neighborhood radius.
- Analyse the a) cluster structure, b) quantization errors, c) topology violations. d) In how far
does this map differ from the very large map trained with a correct/high initial neighborhood
radius?
- Describe and compare the structures found (what is the effect of a „too small“
neighborhood radius? How to detect it?)

In [62]:
# --- Parameters ---
LARGE_DIM = 25          # Same as Task 3
LR_HIGH = 0.7           # Same as Task 3
ITERATIONS = 10000

# --- "Too Small" Neighborhood Radius ---
# A radius < 1.0 means practically no neighbors are pulled along.
# The map acts like k-Means clustering, not a SOM.
SIGMA_TOO_SMALL = 0.5    

print(f"Training Map with Too Small Radius (Sigma={SIGMA_TOO_SMALL})...")

som_bad = MiniSom(LARGE_DIM, LARGE_DIM, data_scaled.shape[1],
                  sigma=SIGMA_TOO_SMALL, learning_rate=LR_HIGH,
                  neighborhood_function='gaussian', random_seed=42)

# Important: Use Random Init to see the twisting effect clearly
# (PCA Init gives it a head start that masks the problem slightly)
som_bad.random_weights_init(data_scaled)
som_bad.train_random(data_scaled, ITERATIONS, verbose=True)

# --- Metrics ---
qe_bad = som_bad.quantization_error(data_scaled)
te_bad = som_bad.topographic_error(data_scaled)

print(f"\n--- Metrics ---")
print(f"Bad Radius Map - QE: {qe_bad:.4f}")
print(f"Bad Radius Map - TE: {te_bad:.4f}")
print("(Compare this TE to the Task 3 Large Map which was ~0.006)")

# --- Prepare for Visualization ---
w_bad = som_bad.get_weights().reshape(LARGE_DIM * LARGE_DIM, data_scaled.shape[1])
sm_bad = SOMToolbox(weights=w_bad, m=LARGE_DIM, n=LARGE_DIM,
                      dimension=data_scaled.shape[1], input_data=data_scaled,
                      classes=classes_array, component_names=component_names_list)

sm_bad._mainview

Training Map with Too Small Radius (Sigma=0.5)...
 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 0.3608704766656835

--- Metrics ---
Bad Radius Map - QE: 0.3609
Bad Radius Map - TE: 0.9760
(Compare this TE to the Task 3 Large Map which was ~0.006)


BokehModel(combine_events=True, render_bundle={'docs_json': {'cbb6f79f-c790-4fef-b07d-fe689ec2f955': {'version…

a.) cluster structure
Visualizations Used: U-Matrix
Instead of the "mountians and valleys" that could be seen before the visualization of the U-matrix of this Som with a too small neighbourhood can best be described by salt and pepper noise with occasional small clusters sprinkled across the map. Interestingly the total QE is significantly lower (0.3609) while the topology error is higher (0.976) than the ones of the previous SOMs. This absence of clusters indicates that due to the low neighbourhood value units overfit the data creating no or very little topological coherence.

b.) quantization errors
Visualizations Used: Quantization Error
As mentioned before the total QE is quite low which is reflected in the Quantization Error map by mostly blue tiles. The units with higher QE are spread randomly across the map and not clustered together. The resulting SOM is closer to a greedy optimization algorithm where every unit is almost free to move to the lowest local error.

c.) topographic violations
Visualizations Used: Neighbourhood Graph (KNN, k=1)
As mentioned in point a the topographic error is close it's maximum 1. This is also reflected in the Neighbourhood graph which is not resembling any structure whatsoever. Edges are drawn randomly. This confirms that while QE is low the low neighbourhood value results numerous topographic violations essentially overfitting the training data.

d.)
described in points a to d: low QE / high TE, Neighbourhood Graph is a mess, U-matrix consists of salt and pepper noise



## 5) Analyze different initial learning rates:
- Train the regular-sized SOM as specified above, but with a (I) much too large / (II) much too
small learning rate (provide justification for the setting of the parameter)
- Analyse for both (I) and (II) a) cluster structure, b) quantization errors, c) topology violations.
d) In how far do these two maps differ from the well-trained map analyzed above?
- Describe and compare the structures found (how can you detect „too small“ learning
rates? When do they start to make sense?

## 6) Analyze different max iterations:
- Train a regular SOM using 2, 5, 10, 50, 100, 1000, 5000, 10000 iterations
- Analyse cluster structure. a) When do cluster structures start to emerge? b) After how many
iterations do they stabilize? c) How can you tell from the quality measures whether the map is
stable? d) Which visualizations help you discover not-yet stable SOM mappings?
- Describe and compare the structures found (what is the effect of a „too low“ number of
iterations, when does it start to converge properly/lead to reasonable structures?)

## 7) Detailed analysis of an „Optimal SOM“:

- Train a SOM using what you consider to be „optimal parameters“ based on sub-tasks 1-6.
- Describe the final model following MLSO.
- Provide a detailed interpretation of the cluster/class structures using a combination of
visualizations and their parameter settings. Describe the findings in detail, specifically
analyzing and providing rationale for:

Cluster densities / cardinalities, shapes: what can you tell about the cluster sizes
shapes, their cardinalities and densities? Can you observe areas of higher/lower
densities? Compare different visualizations that support (or contradict) your hypothesis
and reason/explain why they do so.

Hierarchical cluster relationships: can you detect any hierarchies in the data? How do
they seem to be structured? Which clusters are similar, which are very distant, how
could they be related? Compare different visualizations that support (or contradict)
your hypothesis and reason/explain why they do so.
Topological relations / violations: in which areas can you observe topology violations?

What types of violations do you observe in which areas of the map (i.e. actual violations
due to bad training or the inherent structure of the data vs. cluster data that is mapped
onto the plane). In how far do different visualizations agree on these violations?
Compare different visualizations that support (or contradict) your hypothesis and
reason/explain why they do so.

Class distribution: Which classes are mapped onto which parts of the map? How do
they relate to each other? In how far does the class distribution match the cluster
structure? Which classes are well-separated, which ones less so? What might be the
reason for these overlaps? Is the mapping less correct in these regions (e.g. higher
error measures)? Are these areas well-separated. Which classes form homogeneous
clusters, which form sub-clusters, how similar are these sub-clusters?

Quality of the map in terms of vector quantization and topology violation: is the quality
homogeneous, are there certain areas or classes where the quality of the mapping is
lower, others where it is higher?